In [ ]:
from statistics import mean, median, stdev
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

%reload_ext autoreload
%autoreload 2

In [ ]:
allalgs = ["llsqreg", "ridgereg", "lasso", "sparse_encoded_lasso", "l1l2", "importance", "importance_corfilter", "stepwise", "mlp"]
algs = ["llsqreg", "ridgereg", "lasso", "sparse_encoded_lasso", "l1l2", "importance", "importance_corfilter", "stepwise"]
algnames = ["LLSQ", "Ridge Regression", "LASSO", "SE-Lasso", "L1L2", "Importance", "Importance*", "Stepwise"]
datasets = ["abalone", "adult", "cancer", "card", "covtype", "gene", "glass", "heart", "horse", "madelon", "optdigits", "page-blocks", "pendigits", "poker", "satimage", "segmentation", "shuttle", "soybean", "spect", "thyroid", "vehicle", "waveform"]

In [ ]:
mediandf = []
for alg in allalgs:
    filepart = alg if alg == "mlp" else f"mingood_{alg}"
    df = pd.read_csv(f"../log/performance_{filepart}.txt")[["run", "dataset", "nin", "nout", "time", "ftrain", "ftest", "atrain", "atest"]]
    df = pd.pivot_table(df, index="dataset", aggfunc=median)
    df["alg"] = alg
    mediandf.append(df)
mediandf = pd.concat(mediandf).reset_index()

In [ ]:
ftestdf = pd.pivot(mediandf, index="dataset", columns="alg", values="ftest")[allalgs]
ftestrankdf = ftestdf.rank(axis="columns", method="min", ascending=False)
meanrank = ftestrankdf.aggregate(mean, axis="index").apply(round, args=(2,))
meanrank.name = "mean"
ftestrankdf = ftestrankdf.append(meanrank)
ftestdf = ftestdf[algs].apply(round, args=(3,))
ftestdf.to_csv("exp_mingood_nodes_ftest.csv")
ftestrankdf.to_csv("exp_mingood_nodes_ftestrank.csv")

In [ ]:
timedf = pd.pivot(mediandf, index="dataset", columns="alg", values="time")[allalgs]
timereldf = timedf.apply(lambda s: s / s[0], axis="columns")
meantime = timedf.aggregate(mean, axis="index")
meantime.name = "mean"
meanreltime = timereldf.aggregate(mean, axis="index")
meanreltime.name = "mean"
timedf = timedf.append(meantime)[algs].applymap(lambda t: "%s" % float("%.4g" % t))
timereldf = timereldf.append(meanreltime).applymap(lambda t: "%s" % float("%.4g" % t))
timedf.to_csv("exp_mingood_nodes_time.csv")
timereldf.to_csv("exp_mingood_nodes_timerel.csv")

In [ ]:
timedf.importance_corfilter.sort_values(key=pd.to_numeric)